<a href="https://colab.research.google.com/github/kingsgameBen/iShape_tools/blob/main/reddit_fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 安裝chromedriver 按鍵精靈

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
# wd = webdriver.Chrome('chromedriver',options=options)

### 爬取國外論壇 Reddit - physique phriday 版
判斷是否含有照片，若有就下載貼文及照片連結網站的原始圖片

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from urllib.request import urlopen, Request, HTTPError
from random import choice
import shutil
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
import ssl
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
ssl._create_default_https_context = ssl._create_unverified_context

topic_url_ls = [
"https://www.reddit.com/r/Fitness/comments/izgu3j/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/mrz9ab/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/mnd5co/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/migerf/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/mdkfn6/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/m8d3ou/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/m3ehzr/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ly8opj/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/lsuji4/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/lnca8m/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/li82aq/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ld427j/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/l7ov26/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/kxrkkq/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/kszjmi/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ko8w9o/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ko8w7h/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/kfig7u/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/kb020t/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/k6i75d/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/k1yigm/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/jxmgh6/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/jtejro/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/jp2k9z/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/jku0ok/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/jgkao9/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/jc76v4/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/j7w9ie/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/j3rvf5/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/iv3b3n/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/iqnogb/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/imd9ep/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ii4d96/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/idtzro/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/i9j978/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/i5b5ef/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/i14yx8/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/hombcq/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/hkgfp7/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/hg4si2/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/hby19l/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/h7hxtc/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/gx1r84/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/gsq54j/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/gogkkb/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/gk6cn5/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/gfqf6m/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/gbfdvi/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/g75sby/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/g2z4r0/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/fyd483/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/fu5q88/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/fpudcj/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/flsj0a/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/fhxae2/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/febekx/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/fasjxd/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/f7911p/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/f3q55q/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/f08h5x/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ewmakw/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/et8dug/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/epyh3e/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/emoy7h/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ejdg6l/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/eg9ebx/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ed88gx/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ea255g/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/e6wupn/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/e3cvs8/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/dzyzw4/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/dwopyn/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/dtdc3k/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/dq26du/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/dmv9u3/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/djlbnb/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/dgcyua/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/dd5avw/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/d9y7vi/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/d6spku/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/d3n0t7/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/d0em9c/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/cxeu8j/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/cubsmh/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/cr4aiy/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/co09xf/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/cl24et/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/ci0yv9/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/cf5p1w/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/cc8xfk/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/c9e90s/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/c6idwc/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/c38r31/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/c0ic3w/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/bxssww/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/bv5npw/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/bsf1eo/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/bppc2e/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/bmwftp/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/bk6pna/physique_phriday/",
"https://www.reddit.com/r/Fitness/comments/bhklic/physique_phriday/"
 ]
# print(len(topic_url_ls))
dn = "drive/MyDrive/body_shape/reddit/"
topic_count = 1
for url in topic_url_ls:
    dn_title = url.split("/")[-3]
    print("\ntopic_count:", topic_count, "\ntitle:", dn_title, "\n")
    dir_name = dn + dn_title + "/"
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    req = Request(url)
    header_ls = ["Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36",
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36",
                "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 11.2; rv:87.0) Gecko/20100101 Firefox/87.0",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36 Edg/90.0.818.39"]
    rd_header = choice(header_ls)
    # print(rd_header)
    req.add_header("user-agent", rd_header)
    req.add_header("referer", "https://www.reddit.com/")
    req.add_header("origin", "https://www.reddit.com")
    response = urlopen(req)
    html = BeautifulSoup(response,features="html.parser")
    # print(html)

    comment_blocks = html.find_all("div", attrs={"data-test-id":"comment"})
    # print("comments:", len(comment_blocks))
    comment_count = 1
    for comments in comment_blocks:
        comment_imgs = comments.find_all("a")
        if comment_imgs:
            print("附圖評論數：", comment_count)
            message = comments.find("p").text
            print(message)
            prof_dn = dir_name + str(comment_count) + "/"
            
            # 先去找每個圖片網站超連結的 0 ～ n 張圖片，並下載下來
            img_count = 1
            for img_href in comment_imgs:
                href = img_href["href"]
                # print(href)
                if "ibb" in href:
                    try:
                        response = requests.get(href)
                        html = BeautifulSoup(response.text,features="html.parser")
                        img_container = html.find("div", id="image-viewer-container")
                        if img_container:
                            img_src = img_container.find("img")["src"]
                            print(img_src)
                            if not os.path.exists(prof_dn):
                                os.makedirs(prof_dn)
                            # 看有無圖片，才將評論寫進
                            prof_detail = prof_dn + "info.txt"
                            with open(prof_detail, "w") as txtf:
                                txtf.write(message)
                            fn = prof_dn + str(img_count) + ".jpg"
                            img = requests.get(img_src, stream = True, verify = False)
                            with open(fn, "wb") as f:
                                shutil.copyfileobj(img.raw, f)
                                img_count += 1
                    except HTTPError as e:
                        print("error:", e.code)
                        continue
                elif "imgur" in href:
                    img_src_ls =[]
                    try:
                        driver = Chrome("chromedriver", options=options)
                        driver.get(href)
                        print(href)
                        time.sleep(3)
                        try:
                            # 若存在是否為成人確認頁，則點選確認
                            if len(driver.find_elements(By.CLASS_NAME, 'Wall-choices'))>0:
                                over18 = driver.find_element_by_class_name("btn-wall--yes")
                                ActionChains(driver).move_to_element(over18).click()
                                time.sleep(1)
                            els = driver.find_elements_by_class_name("image-placeholder")
                            # print(els)
                            count = 1
                            for elem in els:
                                link = elem.get_attribute('src')
                                img_src_ls.append(link)
                                # print("="*50)
                                count = count + 1
                        except NoSuchElementException as e:
                            print(e.code)
                            break
                        finally:
                            driver.close()
                        print(len(img_src_ls), img_src_ls)
                        # 看有無圖片，才將評論寫進
                        if img_src_ls:
                            if not os.path.exists(prof_dn):
                                os.makedirs(prof_dn)
                            prof_detail = prof_dn + "info.txt"
                            with open(prof_detail, "w") as txtf:
                                txtf.write(message)
                        for img_src in img_src_ls:
                            fn = prof_dn + str(img_count) + ".jpg"
                            img = requests.get(img_src, stream = True, verify = False)
                            with open(fn, "wb") as f:
                                shutil.copyfileobj(img.raw, f)
                                img_count += 1
                    except HTTPError as e:
                        print("error:", e.code)
                        continue

            # print("="*80, "\n")
            comment_count += 1
    topic_count += 1
